In [1]:
import pandas as pd
import numpy as np
import os

import emoji

import matplotlib.pyplot as plt

import urllib.request

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

from konlpy.tag import Hannanum
hannanum = Hannanum()

print(hannanum.analyze('질문이나 건의사항은 깃헙 이슈 트래커에 남겨주세요.'))

print(hannanum.morphs('질문이나 건의사항은 깃헙 이슈 트래커에 남겨주세요.'))

In [5]:
import konlpy

from konlpy.tag import Kkma
from konlpy.utils import pprint
kkma = Kkma()

In [6]:
print(kkma.nouns('질문이나 건의사항은 깃헙 이슈 트래커에 남겨주세요.'))

['질문', '건의', '건의사항', '사항', '깃헙', '이슈', '트래커']


In [16]:
file = r"/Users/reejungkim/Documents/Git/WebCrawler/GooglePlayStore_Review_FB2021-02-18-17:02:13.csv"

In [17]:
df = pd.read_csv(file, index_col=None)

In [18]:
df.head()

,Unnamed: 0,author,developer_reply,review,review_date,reviewer_ratings,star
0,0,지현우,NaN,한달째 로그인 오류 덕분에 사용하지 못하고 있네요 지인들과 연락도 안되고 이게 뭐...,2021년 2월 14일,51,별표 5개 만점에 1개를 받았습니다.
1,1,감수성art채널,NaN,다른계정 로그인이 안됩니다 본인 비밀번호를 맞게 입력해도 메일주소가 다르게 나오고...,2021년 2월 9일,51,별표 5개 만점에 1개를 받았습니다.
2,2,전지현,NaN,번호가바뀌면서쓰던계정과연동이되지않고잇습니다.로그인코드를문자로보내시는거같은데 이미번...,2021년 1월 22일,51,별표 5개 만점에 1개를 받았습니다.
3,3,윤재형,NaN,"비밀번호를 까먹어서 SMS로 인증코드 확인할려고 ""다음"" 눌렀는데 처음화면으로 돌...",2021년 2월 3일,51,별표 5개 만점에 1개를 받았습니다.
4,4,유진,NaN,제가 이거 한적이 있었는데 깔았다가 지웠거든요 그리고 전화번호도 바꿨어요 그리고 ...,2021년 2월 13일,52,별표 5개 만점에 2개를 받았습니다.


In [19]:
df.shape

(200, 7)

In [20]:
df.review

0       한달째 로그인 오류 덕분에 사용하지 못하고 있네요 지인들과 연락도 안되고 이게 뭐...
1       다른계정 로그인이 안됩니다 본인 비밀번호를 맞게 입력해도 메일주소가 다르게 나오고...
2       번호가바뀌면서쓰던계정과연동이되지않고잇습니다.로그인코드를문자로보내시는거같은데 이미번...
3       비밀번호를 까먹어서 SMS로 인증코드 확인할려고 "다음" 눌렀는데 처음화면으로 돌...
4       제가 이거 한적이 있었는데 깔았다가 지웠거든요 그리고 전화번호도 바꿨어요 그리고 ...
                             ...                        
195                       로그인이 안돼요 그리고 이상한걸로 영정먹고 도움 안돼요
196     페이스북 사진과 가이드라인을 위반 및 이지만 억울합니다 😔😔😔저의 나머지 부계정도...
197     다른 새 계정을 만들려고 했는데 옛날에 잘못 만든 계정을 삭제했는데 계정이 안만들...
198     프로필 업로드가 안돼요.. 몇일째 그래서 다시 깔아보고 이것저것 해보고 버그 신고...
199     페이스북이 자꾸 지멋대로 팅기는 현상이 나타는데 왜이러는거죠? 지우고 다시깔구해도...
Name: review, Length: 200, dtype: object

In [13]:
gdef language_preprocessor(string):
    text =  re.sub('[^a-zA-Z]', ' ', string) #replace non-english with blank
    text = text.lower().split() #decapitalize texts
    text = [word for word in text if not word in stops] #remove stopword
    text = [lm.lemmatize(word) for word in text]  #lemmatization of texts
    return text #' '.join(text)
    

In [14]:
import re

def clean_text(texts):
    corpus = []
    for i in range(0, len(texts)):
        review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',str(texts[i])) #remove punctuation
        review = re.sub(r'\d+','', str(texts[i]))# remove number
        review = review.lower() #lower case
        review = re.sub(r'\s+', ' ', review) #remove extra space
        review = re.sub(r'<[^>]+>','',review) #remove Html tags
        review = re.sub(r'\s+', ' ', review) #remove spaces
        review = re.sub(r"^\s+", '', review) #remove space from start
        review = re.sub(r'\s+$', '', review) #remove space from the end
        corpus.append(review)
    return corpus

df['text processed'] = df['review'].apply(lambda x: clean_text(x)) 

In [15]:
df['nouns'] = df['review'].apply(lambda x: kkma.nouns(x))

In [16]:
df.head()

,Unnamed: 0,author,developer_reply,review,review_date,reviewer_ratings,nouns
0,0,임닉이,NaN,회원가입을하려니까 가입 필수조건을 다 체크하고 전화번호 인증하니까 필수조건 체크가...,2021년 2월 8일,52,"[회원, 가입, 하려, 필수, 필수조건, 조건, 체크, 전화, 전화번호, 번호, 인..."
1,1,Nayeon kang,NaN,나름 대기업인데 일좀 열심히 했으면 좋겠다. 쇼핑앱 여러개 사용중인데 이렇게 뭐 ...,2021년 2월 5일,51,"[나름, 대기업, 일, 쇼핑, 쇼핑앱, 앱, 개, 사용, 사용중, 중, 뭐, 이벤트..."
2,2,산뜻산뜻,NaN,주문조회에서 택배&12개월로 보고 싶은데 택배배송을 누르면 12개월이 3개월로 바...,2021년 1월 25일,51,"[주문, 주문조회, 조회, 택배, 12, 12개월, 개월, 택배배송, 배송, 3, ..."
3,3,신종철,NaN,시스템 잘도 만들어놨네요.왠만해서 귀찮아서 글도 잘 안쓰는 사람인데요. 시스템 왜...,2021년 1월 20일,51,"[시스템, 글, 사람, 식, 이해, 상담, 상담사, 사, 기본, 기본적인, 적인, ..."
4,4,김재규,"<div class=""LVQB0b""><div class=""QoPmEb""></div>...","장바구니에 있는 목록들의 순서를 이동, 변경할 수 있게 해주면 좋겠습니다. 장바구...",2020년 12월 15일,55,"[장, 장바구니, 바구니, 목록, 순서, 이동, 변경, 수, 시간, 물품, 아래, ..."


In [18]:
df['nouns'] = df['nouns'].apply(lambda x: clean_text(x))

In [19]:
df.head()

,Unnamed: 0,author,developer_reply,review,review_date,reviewer_ratings,nouns
0,0,임닉이,NaN,회원가입을하려니까 가입 필수조건을 다 체크하고 전화번호 인증하니까 필수조건 체크가...,2021년 2월 8일,52,"[회원, 가입, 하려, 필수, 필수조건, 조건, 체크, 전화, 전화번호, 번호, 인..."
1,1,Nayeon kang,NaN,나름 대기업인데 일좀 열심히 했으면 좋겠다. 쇼핑앱 여러개 사용중인데 이렇게 뭐 ...,2021년 2월 5일,51,"[나름, 대기업, 일, 쇼핑, 쇼핑앱, 앱, 개, 사용, 사용중, 중, 뭐, 이벤트..."
2,2,산뜻산뜻,NaN,주문조회에서 택배&12개월로 보고 싶은데 택배배송을 누르면 12개월이 3개월로 바...,2021년 1월 25일,51,"[주문, 주문조회, 조회, 택배, , 개월, 개월, 택배배송, 배송, , 개월, 전..."
3,3,신종철,NaN,시스템 잘도 만들어놨네요.왠만해서 귀찮아서 글도 잘 안쓰는 사람인데요. 시스템 왜...,2021년 1월 20일,51,"[시스템, 글, 사람, 식, 이해, 상담, 상담사, 사, 기본, 기본적인, 적인, ..."
4,4,김재규,"<div class=""LVQB0b""><div class=""QoPmEb""></div>...","장바구니에 있는 목록들의 순서를 이동, 변경할 수 있게 해주면 좋겠습니다. 장바구...",2020년 12월 15일,55,"[장, 장바구니, 바구니, 목록, 순서, 이동, 변경, 수, 시간, 물품, 아래, ..."


In [21]:
from collections import Counter
def common_words(df, col, n):
    counts = Counter([item for sublist in df[col] for item in sublist])
    common_words = pd.DataFrame(counts.most_common(n))
    common_words.columns =['Word','count']
    return common_words
    

In [24]:
common_review = common_words(df, 'nouns', 10)
common_review

,Word,count
0,,97
1,앱,56
2,배송,47
3,거,34
4,수,34
5,사용,26
6,주문,25
7,결제,24
8,시간,23
9,상품,21


In [26]:
import plotly.express as px

px.treemap(common_review, 
           path=['Word'], values='count',
           title='Most commonly used words',
          width=400, height=400)

In [35]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

In [48]:
example = "종종 에러가 난다"
stop_words = "종종,가끔"
# 위의 불용어는 명사가 아닌 단어 중에서 저자가 임의로 선정한 것으로 실제 의미있는 선정 기준이 아님

In [49]:
stop_words=stop_words.split(',')

In [50]:
stop_words

['종종', '가끔']

word_tokens = word_tokenize(example)

In [31]:
import hunspell

sent = "한글 맞춤법검사기 재대로작동돼는지테스트"
spelled_sent = spell_checker.check(sent)
checked_sent = spelled_sent.checked
 
print(checked_sent)
 


ModuleNotFoundError: No module named 'hunspell'